In [1]:
import gdsfactory as gf
from blocks import *
from comb_drive_tuning import *
import pandas as pd
import warnings
from dowhen import when
warnings.filterwarnings("ignore")
from pathlib import Path
import sys
sys.path.append(str(Path("../simulation/combdrive_tuning").resolve()))

In [2]:
from doubly_clamped_beam_comb_drive_tuning import doubly_clamped_beam_comb_drive_tuning as f

2026-01-26 16:08:21.995 | WARNING  | kfactory.kcell:show:3958 - Could not connect to klive server


In [3]:
def extract_wg(c:gf.Component):
    c_out = gf.Component()
    polys = c.get_polygons(layers=["WG"], merge=True)[gf.get_layer("WG")]
    if not polys:
        return c_out

    max_poly = max(polys, key=lambda p: p.area())
    c_out.add_polygon(max_poly, layer=gf.get_layer("WG"))
    return c_out

In [9]:
import inspect

when(f, "beam_ref = c << beam").goto("shaft_ref = c.add_ref(shaft)")
when(f,"shaft_ref.connect").do("shaft_ref.move(origin=shaft_ref.center, destination=(0,0))").goto("spring_se = c << spring")
when(f,"beam_pad = U_shape_pad").goto('return c').do(inspect.getsource(extract_wg)).do("c= extract_wg(c)").do("""
c.info['shaft_width'] = shaft_width
""")

In [13]:
params = {
    "shaft_height": 100,
    "beam_length": 1000,
    "finger_pair_num": 10,
    "combdrive_array_num_push": 2,
    "combdrive_array_num_pull": 3,
    "spring_length": 150,
    "spring_loop_num": 1,
    "combdrive_spacing": 100
}
c = f(**params)

In [20]:
-c.info['shaft_width']/2+3/(3+2)*c.info['shaft_width']

61.80000000000001

In [15]:
c.write_gds("shaft_sim.gds",with_metadata=False)

WindowsPath('shaft_sim.gds')

In [21]:
import mph
client = mph.start()
model = client.load(r"C:\Users\weixi\Desktop\Comsol python\NOEMS_Layout\simulation\combdrive_tuning\shaft.mph")
jmodel = model.java

In [22]:
model.parameters()

{'shaft_w': '618[um]',
 'shaft_h': '100[um]',
 'xmin': '-324[um]',
 'xmax': '324[um]',
 'num_pull': '3',
 'num_push': '2',
 'mid': '-shaft_w/2+num_pull/(num_pull+num_push)*shaft_w',
 'spacing': '100[um]'}

In [ ]:
model.parameter("shaft_w", f"{c.info['shaft_width']}[um]")
model.parameter("shaft_h", f"{params['shaft_height']}[um]")
model.parameter("num_pull", str(params['combdrive_array_num_pull']))
model.parameter("num_push", str(params['combdrive_array_num_push']))
model.parameter("spacing", f"{params['combdrive_spacing']}[um]")

In [24]:
model.datasets()

['grav//Solution 1', 'Eigenfreq//Solution 2']

In [26]:
model.evaluate('solid.freq','kHz','Eigenfreq//Solution 2')

array([ 8.6634956 , 16.87152332, 30.50719311, 32.4476913 , 41.66514627,
       47.57785   ])